In [51]:
import math
import torch
from torch import FloatTensor

NAME_INDEX = 0
DATA_INDEX = 1
GRAD_INDEX = 2

In [52]:
def generate_disc_set(nb):
    # Generates data and corresponding target
    
    input = FloatTensor(nb, 2).uniform_(0, 1)
    target = torch.norm(input, 2, 1) < math.sqrt(1/(2*math.pi))
    return input, target.float()

In [53]:
class Parameter():
    # class used to create all the parameters in the same way
    
    def __init__(self, name, tensor, gradient):
        self.name = name       # name of the parameter
        self.data = tensor     # parameter values
        self.grad = gradient   # gradient of the parameter

In [54]:
class Module ( Parameter ) :
    # base module that the following class inherits
    
    def __init__(self):
        super(Parameter, self).__init__()
        self.name = 'Base Module'  # name of the module
        self.param = []            # contains all the parameters of the Module
        
    def forward (self, * input) :
        # Computes the forward pass of the Module.
        # Need to be implemented in future class if one wants to use it.
        raise NotImplementedError
        
    def backward (self , * gradwrtoutput) :
        # Computes the backward pass of the Module for the backpropagation of the loss.
        # Need to be implemented in future class if one wants to use it.
        raise NotImplementedError
        
    def init_parameters ( self ):
        # Initialize the proper parameters for the Module.
        # Need to be implemented in future class if one wants to use it.
        raise NotImplementedError
        
    def add_parameter(self, parameter):
        # Adds the input parameter to the already existing parameters of the Module.
        # Different implementation if the input parameter is:
            # - an object of class 'Parameter'(when you initialize parameters in the current module)
            # - a list of objects(when you add all the parameters of a module to another one)
        if parameter.__class__.__name__ == 'Parameter':
            self.param.append((parameter.name, parameter.data, parameter.grad))
        elif parameter.__class__.__name__ == 'list':                        
            if parameter != []:
                self.param.append(parameter)
                    
    def zero_grad( self ):
        # Reset the gradient of the parameters to 0
        for i in range(len(self.param)):            # loop on the different Module initialized in the 'self' Module
            for j in range(len(self.param[i])):     # loop on the parameters of each Module
                self.param[i][j][GRAD_INDEX][:] = 0
                    
    def optimizer (self, lr = 1e-5):
        # Stochastic Gradient descent
        # updates the parameters in regard of their gradient and the input learning rate
        for i in range(len(self.param)):           # loop on the different Module initialized in the 'self' Module
            for j in range(len(self.param[i])):    # loop on the parameters of each Module
                self.param[i][j][DATA_INDEX][:] -= lr * self.param[i][j][GRAD_INDEX][:]
                
    def parameters ( self ):
        # returns the all parameters of the Module
        return self.param

In [55]:
class MSEloss( Module ):
    # Compute the Mean Square Error between the given input and target
    # no parameters needed
    
    def __init__(self):
        super(MSEloss, self).__init__()
        
    def forward (self, input, target):
        return input.sub(target.view(-1, 1)).pow(2).sum() 
    
    def backward (self, input, target):
        return 2*(input.sub(target.view(-1, 1)))    

In [56]:
class ReLU( Module ):
    # Activation functions: Rectified Linear Unit on each element of the input
    # no parameters needed
    
    def __init__(self):
        super(ReLU, self).__init__()
        self.name = 'ReLU'
        
    def forward(self, input):
        return input.clamp(min = 0)
    
    def backward(self, input, dx):
        input[input <= 0] = 0
        input[input > 0] = 1
        return input * dx

In [57]:
# Module Tanh

class Tanh( Module ):
    # Activation functions: Hyperbolic tangent of each element of the input
    # no parameters needed
    
    def __init__(self):
        super(Tanh, self).__init__()
        self.name = 'Tanh'
        
    def forward(self, input):
        return input.tanh()
    
    def backward(self, input, dx):
        return 4 * (input.exp() + input.mul(-1).exp()).pow(-2) * dx

In [58]:
class Linear( Module ):
    # Linear transformation with certain weights and bias
    
    Linear_counter = 0 # counter of the number of Linear module created in order to name properly the parameters
    
    def __init__(self, input_features, output_features, eps = 1e-2):
        super(Linear, self).__init__()
        self.name = 'Linear'
        Linear.Linear_counter += 1        
        self.init_parameters(input_features, output_features, eps)
        
    def init_parameters (self, input_features, output_features, eps):
        # initializes the weight parameters with a normal distribution and set their gradient to 0
        weigths_name = f'weights{self.Linear_counter}'
        self.weights = Parameter(weigths_name, torch.Tensor(input_features, output_features),
                                 torch.Tensor(input_features, output_features))
        self.weights.data.normal_(0, eps)
        self.weights.grad.zero_()
        
        # initializes the bias parameters with a normal distribution and set their gradient to 0
        bias_name = f'bias{self.Linear_counter}'
        self.bias = Parameter(bias_name, torch.Tensor(output_features), torch.Tensor(output_features))  
        self.bias.data.normal_(0, eps)
        self.bias.grad.zero_()
        
        # adds the weight and bias parameters to this Module
        self.add_parameter(self.weights)
        self.add_parameter(self.bias)
        
    def forward(self, input):
        output = input.mm(self.weights.data)
        output += self.bias.data
        return output
    
    def backward(self, input, ds):
        dx = ds.mm(self.weights.data.t())
        dw = input.t().mm(ds)
        db = ds.t().mm(torch.ones(ds.size(0), 1))
        
        self.weights.grad.add_(dw) # updates the gradient of the parameters with the computed value
        self.bias.grad.add_(db)
        
        return dx

In [59]:
class Sequential( Module ):
    # Sequential container of Modules object.
    # The order is given by the user when creating the object of class Sequential
    
    def __init__(self, *args):
        super(Sequential, self).__init__()
        Linear.Linear_counter = 0               # resets the number of Linear for each sequential object
        self.module_nb = len(args)
        self.fc = [None] * self.module_nb       # contains the modules object in order
        self.x = [None] * (self.module_nb + 1)  # contains the input of the model 
                                                # and the intermediate results after the forward pass of each module 
        
        for id, module in enumerate(args):      # fills the list containing the modules and adds their parameters to
            self.fc[id] = module                # the sequential 'self' module
            self.init_parameters(id)
        
    def init_parameters (self, id):
        self.add_parameter(self.fc[id].parameters())
        
    def forward(self, input):                   # execute the forward pass of each module in the order given by the user
        self.x[0] = input                       # fills the list with the intermediate output of the forward
                                                # pass of each module 
        for i in range(1, self.module_nb + 1):  
            self.x[i] = self.fc[i-1].forward(self.x[i-1])
        return self.x[self.module_nb]
    
    def backward(self, dloss):                  # execute the backward pass of each module in the inverse order
        dx = [None] * (self.module_nb + 1)      # compared to the forward pass
        dx[self.module_nb] = dloss
        
        for i in range(1, self.module_nb + 1):
            j = self.module_nb - i
            dx[j] = self.fc[j].backward(self.x[j], dx[j+1])
            
        return dx

In [25]:
# Net 4 layers

Linear.Linear_counter = 0

class Net4( Module ):
    def __init__(self, hidden_layer):
        super(Net4, self).__init__()
        self.fc1 = Linear(2, 2*hidden_layer)
        self.fc2 = ReLU()
        self.fc3 = Linear(2*hidden_layer, hidden_layer)
        self.fc4 = Linear(hidden_layer, 1)
        self.init_parameters()
        
    def init_parameters ( self ):
        self.add_parameter( self.fc1.parameters() )
        self.add_parameter( self.fc2.parameters() )
        self.add_parameter( self.fc3.parameters() )
        self.add_parameter( self.fc4.parameters() )

    def forward(self, x):
        self.x0 = x
        x = self.fc1.forward( x )
        self.s1 = x
        x = self.fc2.forward( x )
        self.x1 = x
        x = self.fc3.forward( x )
        self.s2 = x
        x = self.fc2.forward( x )
        self.x2 = x
        x = self.fc4.forward( x )
        self.s3 = x
        x = self.fc2.forward( x )
        self.x3 = x
        return x
    
    def backward(self, dloss):
        dx3 = dloss
        ds3 = self.fc2.backward(self.s3) * dx3
        dx2, dw4, db4 = self.fc4.backward( ds3, self.x2 )
        self.fc4.weights.grad.add_(dw4)
        self.fc4.bias.grad.add_(db4)
        ds2 = self.fc2.backward(self.s2) * dx2
        dx1, dw3, db3 = self.fc3.backward( ds2 , self.x1 )
        self.fc3.weights.grad.add_(dw3)
        self.fc3.bias.grad.add_(db3)
        
        ds1 = self.fc2.backward(self.s1) * dx1
        
        dx0, dw1, db1 = self.fc1.backward( ds1 , self.x0 )

        self.fc1.weights.grad.add_(dw1)
        self.fc1.bias.grad.add_(db1)
        
        return dx0

In [60]:
def train_model(model, criterion, train_input, train_target, mini_batch_size = 100, lr = 1e-1, nb_epochs = 250):
    # trains the given model 'nb_epochs' times and prints the loss for each iteration in the end
    
    sum_loss = FloatTensor(nb_epochs).zero_()                     # contains the loss after each iteration
    
    for e in range(0, nb_epochs):
        for b in range(0, train_input.size(0), mini_batch_size):  # divides the input data into batches
            # input go through the model
            output = model.forward(train_input.narrow(0, b, mini_batch_size))
            
            # loss and dloss computation to train the model and visualize it
            loss = criterion.forward(output, train_target.narrow(0, b, mini_batch_size))
            sum_loss[e] += loss
            dloss = criterion.backward(output, train_target.narrow(0, b, mini_batch_size))
            
            # reset the gradient before the backpropagation of the loss by gradient descent
            # in order to update the parameters (learning phase)
            model.zero_grad()
            model.backward(dloss)
            model.optimizer(lr)
            
        print(e, sum_loss[e])

In [77]:
def compute_nb_errors(model, data_input, data_target, mini_batch_size = 100):
    # computes the number of datapoint wrongly classified by the model
    
    total_nb_errors = 0
    
    for b in range(0, data_input.size(0), mini_batch_size):
        
        output = model.forward(data_input.narrow(0, b, mini_batch_size))   # the predicted class is the column of the output with the greater value
        print(output)
        _, predicted_classes = torch.max(output, 1)                        # the predicted class 
        print(predicted_classes)
        
        for k in range(0, mini_batch_size): 
            if data_target[b + k] != predicted_classes[k]:
                total_nb_errors += 1

    return total_nb_errors

In [80]:
train_input, train_target = generate_disc_set(1000)
test_input, test_target = generate_disc_set(1000)
mean, std = train_input.mean(), train_input.std()
train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std);

mini_batch_size = 100
nb_epochs = 250

model = Sequential(Linear(2, 25, 1), ReLU(), Linear(25, 25, 1), ReLU(), Linear(25, 2, 1), ReLU())
criterion = MSEloss()
train_model(model, criterion, train_input, train_target, mini_batch_size, 1e-5, nb_epochs)
nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
nb_train_errors = compute_nb_errors(model, train_input, train_target, mini_batch_size)

print(nb_test_errors)
print('test error Net {:0.2f}%% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                      nb_test_errors, test_input.size(0)))
print('train error Net {:0.2f}%% {:d}/{:d}'.format((100 * nb_train_errors) / train_input.size(0),
                                                      nb_train_errors, train_input.size(0)))


/home/dave/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: other is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.


0 5205.5048828125
1 431.0277099609375
2 311.7295837402344
3 279.97479248046875
4 268.1896667480469
5 262.77606201171875
6 259.81573486328125
7 258.03985595703125
8 256.9345703125
9 256.22821044921875
10 255.76443481445312
11 255.44952392578125
12 255.2233428955078
13 255.04905700683594
14 254.910400390625
15 254.79998779296875
16 254.7117919921875
17 254.63804626464844
18 254.57318115234375
19 254.51605224609375
20 254.46571350097656
21 254.42140197753906
22 254.38233947753906
23 254.3479461669922
24 254.31764221191406
25 254.29095458984375
26 254.26744079589844
27 254.2467498779297
28 254.22850036621094
29 254.21243286132812
30 254.19827270507812
31 254.185791015625
32 254.1748046875
33 254.16513061523438
34 254.1564483642578
35 254.14852905273438
36 254.1413116455078
37 254.1347198486328
38 254.12857055664062
39 254.1227569580078
40 254.1173095703125
41 254.1121826171875
42 254.1073760986328
43 254.10284423828125
44 254.0985870361328
45 254.09457397460938
46 254.0908203125
47 254.087



    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
    0     0
  

-0.5

In [83]:
######################################################################

import torch
import math

from torch import optim
from torch import Tensor
from torch.autograd import Variable
from torch import nn

######################################################################

def generate_disc_set(nb):
    input = Tensor(nb, 2).uniform_(0, 1)
    target = torch.norm(input, 2, 1) < math.sqrt(1/(2*math.pi))
    return input, target

train_input, train_target = generate_disc_set(1000)
test_input, test_target = generate_disc_set(1000)

mean, std = train_input.mean(), train_input.std()

train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)

train_input, train_target = Variable(train_input), Variable(train_target)
test_input, test_target = Variable(test_input), Variable(test_target)

mini_batch_size = 100

######################################################################

def train_model(model, train_input, train_target):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr = 1e-1)
    nb_epochs = 250

    for e in range(0, nb_epochs):
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            optimizer.step()

######################################################################

def compute_nb_errors(model, data_input, data_target):

    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        output = model(data_input.narrow(0, b, mini_batch_size))
        print(output)
        _, predicted_classes = torch.max(output.data, 1)
        for k in range(0, mini_batch_size):
            if data_target.data[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors

######################################################################

def create_shallow_model():
    return nn.Sequential(
        nn.Linear(2, 128),
        nn.ReLU(),
        nn.Linear(128, 2)
    )

######################################################################

model = create_shallow_model()
for p in model.parameters(): p.data.normal_(0, 1e-1)

train_model(model, train_input, train_target)

print('std {:f} train_error {:.02f}% test_error {:.02f}%'.format(
        std,
        compute_nb_errors(model, train_input, train_target) / train_input.size(0) * 100,
        compute_nb_errors(model, test_input, test_target) / test_input.size(0) * 100
    )
    )

######################################################################

RuntimeError: Expected object of type Variable[torch.LongTensor] but found type Variable[torch.ByteTensor] for argument #1 'target'